In [1]:
import datetime
import pandas as pd

In [2]:
df0 = pd.read_feather("data/trip-updates.feather")
df0['datetime'] = df0['time'].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [52]:
df = df0.iloc[:100_000].copy()
df.head()

,trip_id,start_time,start_date,route_id,stop_sequence,delay,time,uncertainty,stop_id,datetime
0,0#785-17,16:39:00,20230506,115,12,46,1683384892,21,20323,2023-05-06 16:54:52
1,0#785-17,16:39:00,20230506,115,13,36,1683384964,21,20215,2023-05-06 16:56:04
2,0#785-17,16:39:00,20230506,115,14,56,1683385076,21,79756,2023-05-06 16:57:56
3,0#785-17,16:39:00,20230506,115,15,90,1683385168,21,72993,2023-05-06 16:59:28
4,0#785-17,16:39:00,20230506,115,16,56,1683385222,21,20295,2023-05-06 17:00:22


In [53]:
# Are there any duplicate rows?
print('Number of duplicate rows: ', df.duplicated().sum())

# Drop duplicates
df.drop_duplicates(inplace=True)

# Are there any duplicate rows?
print('Remaining number of duplicate rows: ', df.duplicated().sum())
df.shape

Number of duplicate rows:  729
Remaining number of duplicate rows:  0


(99271, 10)

In [54]:
weirds = df.groupby(['trip_id', 'start_time', 'uncertainty','start_date']).apply(len)
weirds = weirds.sort_values(ascending=False)
weirds.shape

(8510,)

In [55]:
weirds.describe()

count    8510.000000
mean       11.665217
std        13.952959
min         1.000000
25%         1.000000
50%         5.000000
75%        19.000000
max       159.000000
dtype: float64

In [56]:
w = df[df[weirds.index.names].isin(weirds.index[500]).all(axis=1)].sort_values(by=['datetime'])
w

,trip_id,start_time,start_date,route_id,stop_sequence,delay,time,uncertainty,stop_id,datetime
47773,0#2060-16,18:06:00,20230506,49,7,-362,1683389142,13,75076,2023-05-06 18:05:42
47774,0#2060-16,18:06:00,20230506,49,8,-358,1683389194,13,75077,2023-05-06 18:06:34
47775,0#2060-16,18:06:00,20230506,49,9,-300,1683389292,13,73283,2023-05-06 18:08:12
47776,0#2060-16,18:06:00,20230506,49,10,-280,1683389358,13,73285,2023-05-06 18:09:18
47777,0#2060-16,18:06:00,20230506,49,11,-270,1683389422,13,75078,2023-05-06 18:10:22
47778,0#2060-16,18:06:00,20230506,49,12,-338,1683389424,13,75079,2023-05-06 18:10:24
47780,0#2060-16,18:06:00,20230506,49,14,-304,1683389558,13,75081,2023-05-06 18:12:38
47781,0#2060-16,18:06:00,20230506,49,15,-332,1683389582,13,75082,2023-05-06 18:13:02
47782,0#2060-16,18:06:00,20230506,49,16,-340,1683389646,13,75083,2023-05-06 18:14:06
47783,0#2060-16,18:06:00,20230506,49,17,-326,1683389718,13,78853,2023-05-06 18:15:18
